# PROJET BI

https://www.bil.com/search?find_desc=paris+new+york&find_loc=Paris

In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('inspections-restaurants.csv',sep=';')
arrond=np.unique(data['Code postal'])[:17]
paris=data[(data['Code postal']/1000).astype(int)==75]
#paris['Evaluation'].astype(int).mean


## Extraction des urls

In [2]:
def get_urls():
    data=pd.read_csv('inspections-restaurants.csv',sep=';')
    names=[(n.lower()).replace(' ','+') for n in data['Nom']]
    loc=[loc for loc in data['Code commune']]
    path='https://www.yelp.com/search?'
    urls=[]
    for i in range(len(data)):
        urls.append(path+'find_desc='+names[i]+'&find_loc='+str(loc[i]))
    return urls

def get_bil_urls():
    data=pd.read_csv('inspections-restaurants.csv',sep=';')
    siret=data['SIRET'].values
    urls=[]
    for s in siret:
        url='http://www.bilansgratuits.fr/entreprise/fiche/'+str(s)+'.htm'
        siret=s
        urls.append((url,siret))
    return urls

## Yelp reviews

In [145]:
import nltk
import string
import unicodedata
from BeautifulSoup import BeautifulStoneSoup

def process(txt):
    #txt = txt[txt.find("\n\n"):] # elimination de l'entete (on ne conserve que les caractères après la première occurence du motif
    txt = unicodedata.normalize("NFKD",txt).encode("utf-8","ignore") # elimination des caractères spéciaux, accents...
    punc = string.punctuation    # recupération de la ponctuation
    punc += u'\n\r\t\\'          # ajouts de caractères à enlever
    table =string.maketrans(punc, ' '*len(punc))  # table de conversion punc -> espace
    txt = string.translate(txt,table).lower() # elimination des accents + minuscules
    return txt

def conv_html(htmlstring):
    return BeautifulStoneSoup(htmlstring,
                   convertEntities=BeautifulStoneSoup.HTML_ENTITIES).contents[0]


yelp=pd.read_csv('yelp_revs.csv',sep=';',encoding='utf-8')
#print process(conv_html(yelp['comment'].loc[2]))
yelp['revs']=yelp['comment'].apply(lambda s: process(conv_html(s)))

y=yelp[yelp['revs'].str.contains('sale')]['revs']
yb=[yi for yi in y if(not np.where('sale' in yi))]
yb

[]

## etalages_et_terrasses

In [105]:
import numpy as np
import pandas as pd
etal=pd.read_csv('etalages_et_terrasses_autorisees_a_paris.csv',sep=';')
etal.head()

,dossier,arrondissement,adresse,lieu,libelle_type_objet,dima,dimb,dosred,Coordonnées
0,793 937 093 002,75020,93 AVENUE GAMBETTA,A,TERRASSE OUVERTE,1.50,2.00,SARL CEP 20 (BRASSERIE),"48.866456, 2.399652"
1,793 937 125 003,75020,Pan coupé 125 AVENUE GAMBETTA / 120 RUE ORFILA,A,TERRASSE OUVERTE,2.67,4.26,SARL LA POINTE AUX PIMENTS (RESTAURANT),"48.868115, 2.40109"
2,793 939 002 002,75020,202 RUE DES PYRENEES,B,TERRASSE FERMEE,1.30,12.05,SNC INDIANA GAMBETTA (RESTAURANT),"48.864386, 2.39905"
3,793 939 007 003,75020,7 PLACE GAMBETTA,A,TERRASSE OUVERTE,0.60,8.20,SA MC DONALD S FRANCE (RESTAURATION RAPIDE - ...,"48.864778, 2.397904"
4,793 939 010 002,75020,206 RUE DES PYRENES,B,TERRASSE OUVERTE,0.70,3.30,M ROMIEU (BRASSERIE),"48.865147, 2.398322"


### ajout d'une nouvelle colonne indiquant le libelle

In [247]:
etal['libelle']=etal.dosred.apply(lambda s:s[s.find("(")+1:s.find(")")]).values
etal['DOSRED_MOD']=etal.dosred.apply(lambda s:s[:s.find("(")]+s[s.find(")")+1:]).values
#etal.head()

### Extraction des restaurants uniquement

In [235]:
#Extraction des restaurants parmi toutes les données
restos=etal[etal['dosred'].str.contains('RESTA')]
brass=etal[etal['dosred'].str.contains('BRASSERIE')]
pizz=etal[etal['dosred'].str.contains('PIZZERIA')]
all_rest=restos.append(brass).append(pizz)
all_rest.to_csv('etal_rests.csv',sep=';')

all_rest=all_rest[~all_rest['Coordonnées'].isnull()]

sanit=pd.read_csv('inspections-restaurants.csv',sep=';')
sanit.head()
sanit['Coordonnées géographiques']=sanit['Coordonnées géographiques']=sanit['Coordonnées géographiques'].apply(lambda x: np.array(x.split(','),dtype='float'))
all_rest['Coordonnées']=all_rest['Coordonnées'].apply(lambda x: np.array(x.split(','),dtype='float'))
sanit['Coordonnées géographiques']
all_rest['dist']=99999.
loc=4
a=sanit['Coordonnées géographiques'].loc[loc]
#print a
#dist=[np.linalg.norm(x-a) for x in all_rest['Coordonn\xc3\xa9es']]
#m,argm=np.min(dist),np.argmin(dist)
#print m
#print sanit.loc[loc]
#print ''
#print all_rest.loc[argm]
#all_rest['dist']=all_rest['dist'].apply(lambda x:np.linalg.norm(a-x))
#all_rest[:20]
#np.linalg.norm()
#for i in range(len(sanit)):
    #coord=sanit['Coordonnées géographiques'].loc[i]
#    a,b=sanit['Coordonnées géographiques'].values[i].split(',')
#    print ','.join([a[:9],b[:8]])
#    pos=','.join([a[:9],b[:8]])
#    print all_rest[all_rest['Coordonnées'].str.contains(pos)].values

## Web crawler pour extraire les bilans à partir du n de SIRET

In [ ]:
# -*- coding: utf-8 -*-
from BeautifulSoup import BeautifulSoup
from urlparse import urljoin
import urllib2
import argparse
import re
import codecs
import time
import random
import biurls
import pandas as pd

get_bil_page = lambda page: biurls.get_bil_urls()[page][0]


def crawl_page(page, verbose=False):

    try:
        page_url = get_bil_page(page)
        soup = BeautifulSoup(urllib2.urlopen(page_url).read())

    except Exception, e:
        print str(e)
        return []
    data={}
    rest=soup.findAll('h2',attrs={'class':"title accroche"},text='Informations Générales'.decode('utf-8'))
    rest=rest[0].next.next
    fj=''
    cap=''
    nbetab=''
    date=''
    data['url']=page_url
    try:
        fj=rest.find('td',{'class':'leftElement large1'},text='Forme juridique').next.next.next
        data['forme juridique']=fj
    except Exception, e:
        if verbose: print 'extract fail', str(e)
    try:
        cap=rest.find('td',{'class':'leftElement large1'},text='Capital').next.next.next
        cap=cap.split('&nbsp;&euro;')[0]
        data['capital']=cap
    except Exception, e:
        if verbose: print 'extract fail', str(e)
    try:
        nbetab=rest.find('td',{'class':'leftElement large1'},text='Nombre d\'établissements'.decode('utf-8')).next.next.next
        data['nombre etabl']=nbetab
    except Exception, e:
        if verbose: print 'extract fail', str(e)
    try:
        date=rest.find('td',{'class':'leftElement large1'},text='Création'.decode('utf-8')).next.next.next
        data['date creation']=date
    except Exception, e:
        if verbose: print 'extract fail', str(e)

    
    return data


urls=biurls.get_bil_urls()
results=[]
for index in range(len(urls)):
    print('index: '+str(index))
    rest=crawl_page(index)
    rest['page']=index
    rest['siret']=urls[index][1]
    results.append(rest)
    #print rest
    print 'page crawled'
    time.sleep(random.randint(1, 3) * .931467298)
df=pd.DataFrame(results,index_col=False)
df.to_csv('siret.csv',sep=';',index_col=True)


In [9]:
df=pd.DataFrame(results)
df.to_csv('siret.csv',sep=';',index_col=False)

## For google Maps

In [ ]:
paris_rest=data[(data['Code postal']/1000).astype(int)==84]
pr=paris_rest[['Coordonnées géographiques','Nom','Evaluation']]
res=[]
for i in range(len(pr)):
    s=pr.values[i]
    s[1]=s[1].replace('\'','\\\'')
    res.append('['+s[0]+',\''+s[1]+', Evalution: '+str(s[2])+'\''+']')
print ',\n'.join(res)